In [1]:
import random

In [43]:
import numpy as np

In [50]:
import math

In [101]:
import time

In [23]:
def Random_probabilies(dices, probabilities):
    prob = [x/sum(probabilities) for x in probabilities]
    rolldice = random.uniform(0,1)
    initial  = 0
    for dice, probability in zip(dices,prob):
        initial += probability
        if  rolldice < initial:
            break
    return dice

In [29]:
result = []
for i in range(100000):
    result.append(Random_probabilies(dices, probabilities))
print(result.count(1),result.count(2),result.count(3),result.count(4),result.count(5),result.count(6))

1596 3126 6381 12682 25366 50849


In [142]:
Dna = ['CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA',
       'GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG',
       'TAGTACCGAGACCGAAAGAAGTATACAGGCGT',
       'TAGATCAAGTTTCAGGTGCACGTCGGTGAACC',
       'AATCCACCAGCTCCACGTGCAATGTTGGCCTA']
k = 8
t = 5
N = 100

In [31]:
def initialMotifs(Dna,k):
    Motifs = []
    for String in Dna:
        index = random.randint(0,(len(String)-k))
        Motifs.append(String[index:index+k])
    return Motifs

In [44]:
def ProfileMotif(Motif):
    row = len(Motif)
    column = len(Motif[0])
    matrix = []
    for i in range(row):
        matrix += list(Motif[i])
    matrix = np.array(matrix).reshape(row,column)
    profile = np.zeros((4,column))
    for i in range(column):
        profile[:,i][0] = (np.sum(matrix[:,i]=='A')+1)/(row+4)
        profile[:,i][1] = (np.sum(matrix[:,i]=='C')+1)/(row+4)
        profile[:,i][2] = (np.sum(matrix[:,i]=='G')+1)/(row+4)
        profile[:,i][3] = (np.sum(matrix[:,i]=='T')+1)/(row+4)
    return profile

In [46]:
def Pr(Pattern, ProfileMotif):
    Symbol = ['A','C','G','T']
    probability = 1.0
    for i in range(len(Pattern)):
        probability *= float(ProfileMotif[:,i][Symbol.index(Pattern[i])])
    return probability

In [47]:
def ProfileRandomlyGeneratedkmer(Genome, k, Prof):
    kmer = []
    probable = []
    for i in range(len(Genome)-k+1):
        kmer.append(Genome[i:i+k])
        probable.append(Pr(Genome[i:i+k],Prof))
    return Random_probabilies(kmer, probable)

In [45]:
def Entropy(column):
    result  = 0
    for i in column:
        if i == 0:
            result += 0
        else:
            result += -i*math.log2(i)
    return result

def Score(motifs):
    result = 0
    eachEntropy = []
    for i in range(len(motifs[0])):
        result += Entropy(motifs[:,i])
        eachEntropy.append(Entropy(motifs[:,i]))
    return result

In [64]:
def Hamdist(p,q):
    if len(p) != len(q):
        return -1
    k = 0
    for i in range(len(p)):
        if p[i] != q[i]:
            k += 1
    return k
def Consensus(Motifs):
    matrix = ProfileMotif(Motifs)
    Symbol = ['A','C','G','T']
    consensus = ''
    for i in range(len(matrix[0])):
        consensus += Symbol[matrix[:,i].argmax()]
    return consensus

def Score2(motifs):
    result = 0
    Pattern = Consensus(motifs)
    for i in range(len(motifs)):
        result += Hamdist(motifs[i],Pattern)
    return result

In [211]:
def MaximumFinding(skew):
    i = 0
    index = []
    while max(skew) in skew[i:]:
        j = skew[i:].index(max(skew))
        i += j+1
        index.append(i-1)
    return index

In [209]:
def ProfileMostProbablekmer(Genome, k, ProfileMotif):
    probable = []
    Pattern = []
    for i in range(len(Genome)-k+1):
        probable.append(Pr(Genome[i:i+k],ProfileMotif))
    index = MaximumFinding(probable)
    for j in index:
        Pattern.append(Genome[j:j+k])
    return Pattern[0]

In [207]:
def MotifsProfile(Profile, Dna):
    Motifs = []
    k = len(Profile[0])
    for Genome in Dna:
        Motifs.append(ProfileMostProbablekmer(Genome, k, Profile))
    return Motifs

In [205]:
def RandomizedMotifSearch(Dna, k, t):
    Motifs = initialMotifs(Dna,k)
    BestMotifs = Motifs
    while 1:
        profile = ProfileMotif(Motifs)
        Motifs = MotifsProfile(profile,Dna)
        if Score(ProfileMotif(Motifs)) < Score(ProfileMotif(BestMotifs)):
        #if Score2(Motifs)<Score2(BestMotifs):
            BestMotifs = Motifs  #
        else:
            return BestMotifs

In [73]:
def GibbsSampler(Dna, k, t, N):
    Motifs = initialMotifs(Dna, k)
    BestMotifs = Motifs
    for j in range(N):
        i = random.randint(0,t-1)
        Motifs.pop(i)
        Prof = ProfileMotif(Motifs)
        kmer = ProfileRandomlyGeneratedkmer(Dna[i], k, Prof)
        Motifs.insert(i,kmer)
        #if Score(ProfileMotif(Motifs)) < Score(ProfileMotif(BestMotifs)):
        if Score2(Motifs) < Score2(BestMotifs) :
            BestMotifs = Motifs
    return BestMotifs

In [214]:
def CombinedGibbsSamplerWithRandomizedMotifSearch(Dna, k, t, N):
    Scorex = float('inf')
    for i in range(30):
        Motifs = RandomizedMotifSearch(Dna, k, t)
        if Score2(motifs) < Scorex:
            Scorex = Score2(motifs)
            BestMotifs = Motifs
    for j in range(N):
        i = random.randint(0,t-1)
        Motifs.pop(i)
        Prof = ProfileMotif(Motifs)
        kmer = ProfileRandomlyGeneratedkmer(Dna[i], k, Prof)
        Motifs.insert(i,kmer)
        #if Score(ProfileMotif(Motifs)) < Score(ProfileMotif(BestMotifs)):
        if Score2(Motifs) < Score2(BestMotifs) :
            BestMotifs = Motifs
    return BestMotifs

In [213]:
start = time.clock()
BestMotifs = RandomizedMotifSearch(Dna, k, t)
end = time.clock()
print("run: %fs" % (end-start))

run: 0.225065s


In [197]:
info = []
with open('/Users/gaopeng/Documents/bioinformatics/data/dataset_163_4.txt') as f:
    for line in f.readlines():
        info.append(line.strip())
k = int(info[0].split()[0])
t = int(info[0].split()[1])
N = int(info[0].split()[2])
Dna = info[1:]

In [216]:
start = time.clock()
Scorex = float('inf')
BestMotifs = CombinedGibbsSamplerWithRandomizedMotifSearch(Dna, k, t, N)
end = time.clock()
print("run: %fs" % (end-start))

run: 17.129837s


In [222]:
start = time.clock()
Scorex = float('inf')
for i in range(3):
    motifs = GibbsSampler(Dna, k, t, N)
    if Score2(motifs) < Scorex:
        Scorex = Score2(motifs)
        BestMotifs = motifs
end = time.clock()
print("run: %fs" % (end-start))
Score2(BestMotifs)

run: 25.122671s


87

In [220]:
Score2(BestMotifs)

65

In [203]:
start = time.clock()
Scorex = float('inf')
for i in range(10):
    motifs = GibbsSampler(Dna, k, t, N)
    if Score2(motifs) < Scorex:
        Scorex = Score2(motifs)
        BestMotifs = motifs
end = time.clock()
print("run: %fs" % (end-start))

run: 85.496146s


In [204]:
r = ''
for i in BestMotifs:
    r += i+'\n'
print(r,Score2(BestMotifs))

GGTTTAGACCTTAGG
AGTACCTTCATTGGC
ACACAGTTCATTGGC
AGTTATAACATTGGC
TATTAGTTCATTGGA
TTCTAGTTCATTGGC
AGTTAGACAATTGGC
AGTTAGTATGTTGGC
AGTGTCTTCATTGGC
AGCGTGTTCATTGGC
AGTTAGTTCGGGGGC
AGTTAGTTCATGACC
GGTTAGTTCATTGTA
AGTTGAATCATTGGC
AGTTACCGCATTGGC
AGTTAGTTACATGGC
AGTTAGTTCATTCTG
AGTTAGAAAATTGGC
AGTTAGTTCAGGCGC
AGTTTCGTCATTGGC
 65


In [66]:
Score2(['TCTCGGGG','CCAAGGTG','TACAGGCG','TTCAGGTG','TCCACGTG'])

9

In [223]:
info = []
with open('/Users/gaopeng/Documents/bioinformatics/data/DosR.txt') as f:
    for line in f.readlines():
        info.append(line.strip())
Dna = info
t = len(Dna)
len(Dna[0])

250

In [224]:
k = 20
N = 200
start = time.clock()
BestMotifs = GibbsSampler(Dna, k, t, N)
end = time.clock()
print("run: %fs" % (end-start))
Score2(BestMotifs)

run: 0.843938s


69

In [230]:
k = 20
N = 200
start = time.clock()
Scorex = float('inf')
for i in range(10):
    motifs = GibbsSampler(Dna, k, t, N)
    if Score2(motifs) < Scorex:
        Scorex = Score2(motifs)
        BestMotifs = motifs
end = time.clock()
print("run: %fs" % (end-start))
print(Score2(BestMotifs))
Consensus(BestMotifs)

run: 8.297413s
55


'CGGGACCTACGTCCCTAGCC'

In [226]:
Score2(BestMotifs)

62